In [ ]:
import numpy as np
import pandas as pd
import os
import warnings
import librosa
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
from random import shuffle
import scipy.stats as st

from pynwb import NWBHDF5IO
from pynwb.image import ImageSeries
from pynwb.epoch import TimeIntervals

In [ ]:
nwbpath = 'C:\\Users\\nhogl\\Documents\\GitHub\\scn2a-paper-GWJSNH-code\\NWB'

assay = 'reunion'
allnwb = []
allnwb += [os.path.join(nwbpath,each) for each in os.listdir(nwbpath) if assay in each]

# usefield = 'single_behavior_Huddle'

In [ ]:
# compare huddle length of partner vs stranger per group
# distributions per individual
# collect all durations for partner and stranger for each individual and put those in a dataframe
# can add histograms to dataframe
# show paired medians per individual
# collapse into a single distribution? per group

# ID, sex, GT, dur, partner, stranger, time


# loop over files
# make a table of specified info
# add to big table

for i,fname in enumerate(allnwb):
    # open file
    io = NWBHDF5IO(os.path.join(nwbpath,fname), mode="r") # <-- open in write to append snippets?
    nwbfile = io.read()
    
    # for sanity check, print out the name of the file being handled
    pathsplit = fname.split('\\')
    recstem = pathsplit[-1].strip('.nwb')
    print(recstem)
    
    # filter out files that don't have complete timelines
    
    try:
        partnerdf = nwbfile.intervals['single_behavior_Huddle Partner'].to_dataframe()
        nrpartner = partnerdf.shape[0]
        masspartner = ['partner']*nrpartner
    except:
        print('No partner huddles')
    try:
        strangerdf = nwbfile.intervals['single_behavior_Huddle Stranger'].to_dataframe()
        nrstranger = strangerdf.shape[0]
        massstranger = ['stranger']*nrstranger
    except:
        print('No stranger huddles')
        
    # make new dataframe with start times, durations, and labels
    
    combdf = pd.concat([partnerdf, strangerdf])
    
    nrentries = combdf.shape[0]
    
    massGT = [GT]*nrentries
    massPairtag = [pair_tag]*nrcalls
    massSex = [nwbfile.subject.sex]*nrcalls
    
    nrcols = statsdf.shape[1]
    
    # add columns
#     statsdf.insert(nrcols, 'Experiment', massExpt, True)
#     statsdf.insert(nrcols+1, 'Pairtag', massPairtag, True)
#     statsdf.insert(nrcols+2, 'RecSess', massRecSess, True)
#     statsdf.insert(nrcols+3, 'GT', massGT, True)